В этой задаче мы научимся строить решающие деревья с нуля. Это несложный, но распространённый алгоритм машинного обучения. Деревья применяются для решения задач регрессии, классификации, аплифт-моделирования и ранжирования.

Решающие деревья – составной элемент в ансамблях моделей: случайный лес, градиентный бустинг.

Решающие деревья реализованы во многих популярных библиотеках. Однако ничто не может быть лучшим знакомством с ними и с механизмом их работы, как возможность один раз написать решающее дерево самому. Понимание этих принципов поможет как в реальной работе, так и на собеседованиях – особенно в случаях, когда речь пойдет о принципах работы упомянутого градиентного бустинга.

# Что такое решающее дерево?
Решающее дерево – это бинарное дерево, в узлах которого содержатся условия вида "возраст < 30" (для численных признаков) или "пол = мужской" (для категориальных), а в листьях предсказание модели (класс в случае классификации или численное значение в случае регрессии). 

# Задача регрессии
Мы будем предсказывать количество дней просрочки. Клиент подает заявку на кредит, указывает ряд данных о себе, а также сумму и срок кредита. Наша задача предсказать количество дней просрочки. Для большинства клиентов это значение равно 0 дней. Но для некоторых клиентов это значение больше нуля. Чем выше прогнозное значение "дней просрочки", тем более рискованно для банка одобрить заявку на кредит.



Для обучения алгоритма у нас есть размеченный датасет с заявками на кредит. Целевая переменная — delay_days. Наша задача обучить модель (решающее дерево), которая для новых клиентов будет предсказывать количество дней просрочки, чтобы среднеквадратичная ошибка была минимальной.

Дерево строится по данным обучающей выборки, на которой мы знаем значение признаков и целевую переменную для каждого объекта. Затем на новых данных с помощью построенного дерева выполняется предсказание целевой переменной, как мы это делали для примера с Андреем и Петром. Только вместо "одобрить/ не одобрить" мы будем предсказывать количество дней.

```sql
SELECT
    age,
    income,
    dependents,
    has_property,
    has_car,
    credit_score,
    job_tenure,
    has_education,
    loan_amount,
    dateDiff('day', loan_start, loan_deadline) AS loan_period,
    CASE
        WHEN loan_payed > loan_deadline
        THEN dateDiff('day', loan_deadline, loan_payed)
        ELSE 0
    END AS delay_days
FROM
    default.loan_delay_days
ORDER BY
    id;

```

# Информационный критерий
Информационный критерий — это метод оценки качества разбиения данных на две или более группы в решающем дереве. Это условие, по которому мы выбираем оптимальное разбиение данных в каждом узле дерева. Хороший информационный критерий помогает выбрать такой вариант разбиения выборки на две подвыборки, чтобы ветвление дерева наиболее осмысленно разбивало данные.

Предсказание в задаче регрессии выполняется по среднему значению целевой переменной всех объектов, попавших в лист на этапе обучения. В первом примере среднее значение для листа слева и среднее значение для листа справа будет ближе к истинным значениям. Во втором примере среднее значение будет дальше.

# Критерий разбиения
Для задач регрессии в качестве критерия разбиения обычно используется MSE (Mean Squared Error). MSE – это средний квадрат ошибки (разницы) между фактическими значениями целевой переменной и предсказанными. Мы считаем MSE до разбиения и после. То разбиение, которое даст наибольшее снижение ошибки – оптимальное.

Для выбора варианта разбиения оценивают средневзвешенное значение критерия в обоих выборках. Если средневзвешенное значение критерия в выборках слева и справа лучше, значит разбиение хорошее.

# Задание
Реализуйте две функции:

mse – принимает на вход вектор значений, возвращает значение mse.
weighted_mse – принимает на вход два вектора значений (слева и справа) и возвращает средневзвешенную mse.

MSE – это ошибка между истинными значениями и предсказанными. Как мы разобрали выше, предсказание в задаче регрессии – это среднее целевой переменной для всех объектов узла. 

Поэтому в качестве предсказанного значения вычислите среднее значение выборки. И найдите MSE между истинными значениями выборки и предсказанным (средним).

In [ ]:
import numpy as np

def mse(y: np.ndarray) -> float:
    """Compute the mean squared error of a vector."""
    y_mean = np.mean(y)
    return np.mean((y - y_mean) ** 2)


def weighted_mse(y_left: np.ndarray, y_right: np.ndarray) -> float:
    """Compute the weighted mean squared error of two vectors."""
    mse_left = mse(y_left)
    mse_right = mse(y_right)
    total_len = len(y_left)+len(y_right)
    return (mse_left*len(y_left)+mse_right*len(y_right))/total_len


# Сплит по одному признаку
Датасет подготовлен. Информационный критерий считать умеем. 

Теперь научимся определять, какой сплит (разбиение) будет оптимальным. На этом шаге мы делаем сплит только по одному признаку. Выбираем такое значение признака, чтобы средневзвешенный MSE был минимальынм.

# Задание
Реализуйте функцию split. Функция принимает на вход выборку (матрицу признаков X и вектор целевой переменной y) и индекс признака. Возвращает значение порога с наилучшим разбиением.

Например, если разбиение выполняется по полю age, в качестве аргумента feature будет передано значение 0. В качестве результата может быть значение 42. Что значит, что все объекты с age <= 42.0 попадают в левую выборку, остальные в правую выборку.

Перебираем все значения признака:

    Разделяем данные по порог -> подвыборка слева, подвыборка справа
        Считаем взвешенный критерий для подвыборок слева и справа
Возвращаем значение порога, который дает минимальное значение взвешенного критерия


In [1]:
def split(X: np.ndarray, y: np.ndarray, feature: int) -> float:
    """Find the best split for a node (one feature)"""
    my_x = X[:, feature]
    best_threshold = None
    min_mse = float('inf')

    for x in np.unique(my_x):
        left_mask = my_x <= x
        right_mask = my_x > x
        
        y_left = y[left_mask]
        y_right = y[right_mask]
        
        mse_ = weighted_mse(y_left, y_right)
        
        if mse < min_mse:
            min_mse = mse
            best_threshold = x

    return best_threshold

# Сплит по всем признакам
Отлично, мы научились разбивать выборку на две подвыборки по одному признаку, выбирая значения признака таким образом, чтобы минимизировать средневзвешенный критерий MSE.

Теперь сделаем то же самое, но для всех признаков. Ваша задача — найти наилучший сплит среди всех признаков.

# Задание
Реализуйте функцию best_split. Функция принимает на вход выборку (матрицу признаков X и вектор целевой переменной y). Возвращает индекс признака и значение порога с наилучшим разбиением.

In [ ]:
from __future__ import annotations

import numpy as np


def best_split(X: np.ndarray, y: np.ndarray) -> tuple[int, float]:
    """Find the best split for a node (one feature)"""
    bests = {}
    
    for i in range(X.shape[1]): 
        bests[i] = split(X, y, i)
        
    best_threshold = min(bests.values())  
    best_feature = min(bests, key=bests.get)  

    return best_feature, best_threshold

Теперь реализуем класс Node. Это вспомогательный класс. Каждый экземпляр класса описывает отдельный узел или лист в дереве. Из этих нод мы и будем строить дерево.


Давайте подумаем, какие атрибуты и каких типов необходимо добавить для класса Node:

feature – индекс признака, по которому выборка, попавшая в ноду, разделяется на 2 дочерние ноды. Тип: int
threshold – значение порога. Все элементы, у которых значение признака feature <= threshold, попадают в левую дочернюю ноду. Остальные в правую дочернюю ноду. Тип: float
n_samples – количество объектов в ноде. Тип: int
value – среднее значение целевой переменной среди всех объектов в ноде, округленное до целого. Тип: int
mse – значение критерия MSE в ноде. Тип: float
left – левая дочерняя нода. Тип: Node
right – правая дочерняя нода. Тип: Node
Для листьев также будем использовать класс Node. Только поля feature, threshold, left, right не будут заполнены.

Задание
Вам дана заготовка класса Node с тремя заполненными атрибутами. Завершите класс полностью, добавив все оставшиеся атрибуты.

Обратите внимание, что класс Node мы реализуем как датакласс. Это более удобная и компактная запись. Если вам не требуется реализовывать дополнительную логику при инициализации экземпляра класса, то используйте датаклассы. Ваш код будет выглядет более читабельным. Подробнее о датаклассах можно почитать по ссылке.

Для всех атрибутов укажите значение None в качестве дефолтного. Кажется, для числовых атрибутов это выглядит странным и можно использовать дефолтный 0. Но дефолтный 0 может совпасть с значением индекса признака, значением порога или mse. Поэтому более универсально для не заполненных значений использовать None. 

In [1]:
from __future__ import annotations

from dataclasses import dataclass


@dataclass
class Node:
    """
    A node in a decision tree.

    Parameters
    ----------
    feature : int, optional (default=None)
        The feature index used for splitting the node.
    threshold : float, optional (default=None)
        The threshold value at the node.
    n_samples : int, optional (default=None)
        The number of samples at the node.
    value : int, optional (default=None)
        The value of the node (i.e., the mean target value of the samples at the node).
    mse : float, optional (default=None)
        The mean squared error of the node (i.e., the impurity criterion).
    left : Node, optional (default=None)
        The left child node.
    right : Node, optional (default=None)
        The right child node.
    """
       
    feature: int = None
    threshold: float = None
    n_samples: int = None
    value: int = None
    mse: float = None
    left: Node = None
    right: Node = None
    

DecisionTreeRegressor
Подведем промежуточный итог, чему мы уже научились:

1. Считать информационный критерий и взвешенный критерий, чтобы оценить качество сплита.
2. Находить наилучшее разбиение выборки.
3. Определили сущность нода и описали ее классом Node.
# Пора переходить к построению дерева.

Как строится дерево?
Для построение дерева используется рекурсивный алгоритм. Корневая нода делится на две дочерние ноды. Каждая дочерняя нода в свою очередь делится еще на две дочерние ноды. Деление продолжается, пока не будет достигнут критерий остановки. В рекурсивных алгоритмах это еще называется базовый случай.

Разделить_ноду (Node):
    Если достигли критерия остановки, то останавливаемся.
    Ищем лучшее разбиение:
        Если нашли лучшее разбиение, разбиваем Node на две дочерние Left и Right:
            Разделить_ноду(Left)
            Разделить_ноду(Right)
        Иначе останавливаемся.


Критерии остановки
На практике используется много критериев остановки. Мы реализуем только два:

- max_depth (максимальная глубина дерева) — если достигли максимальной глубины, нода дальше не делится и остается листом.
- min_samples_split (минимальное число объектов в ноде для дальнейшего деления) — если объектов меньше, нода дальше не делится и остается листом.

Depth-wise подход
В этой задаче мы используется подход построения дерева, который называется depth-wise (в глубину). Его идея в том, что каждая нода делится независимо до тех пор, пока не будет достигнут один из критериев остановки. Дерево очень быстро растет в глубину. Причем растет достаточно равномерно. Такой подход по умолчанию используется, например, при построении деревьев в популярной библиотеке XGBoost.

# Задание
Реализуйте класс DecisionTreeRegressor.

Метод fit — это публичный метод класса (api класса). Он предназначен для использования внешним кодом, который создает экземпляры класса и манипулируют ими.

Методы, названия которых начинаются с символа "_", называются внутренними методами. Они реализуют внутреннюю логику класса и не предназначены для использования снаружи.

Перенесите код из реализованных ранее функций: mse, weighted_mse, best_split в одноименные внутренние методы. Перенести класс Node.

Реализуйте внутренний метод _split_node

In [22]:
import json
from __future__ import annotations
from dataclasses import dataclass
import numpy as np


@dataclass
class Node:
    """
    A node in a decision tree.

    Parameters
    ----------
    feature : int, optional (default=None)
        The feature index used for splitting the node.
    threshold : float, optional (default=None)
        The threshold value at the node.
    n_samples : int, optional (default=None)
        The number of samples at the node.
    value : int, optional (default=None)
        The value of the node (i.e., the mean target value of the samples at the node).
    mse : float, optional (default=None)
        The mean squared error of the node (i.e., the impurity criterion).
    left : Node, optional (default=None)
        The left child node.
    right : Node, optional (default=None)
        The right child node.
    """

    feature: int = None
    threshold: float = None
    n_samples: int = None
    value: int = None
    mse: float = None
    left: Node = None
    right: Node = None


@dataclass
class DecisionTreeRegressor:
    """Decision tree regressor."""
    max_depth: int
    min_samples_split: int = 2

    def fit(self, X: np.ndarray, y: np.ndarray) -> DecisionTreeRegressor:
        """Build a decision tree regressor from the training set (X, y)."""
        self.n_features_ = X.shape[1]
        self.tree_ = self._split_node(X, y)
        return self

    def _mse(self, y: np.ndarray) -> float:
        """Compute the mse criterion for a given set of target values."""
        return np.mean((y - np.mean(y)) ** 2)

    def _weighted_mse(self, y_left: np.ndarray, y_right: np.ndarray) -> float:
        """Compute the weighted mse criterion for two given sets of target values."""
        if y_left.size == 0 or y_right.size == 0:
            return float('inf')
        num = self._mse(y_left) * y_left.size + self._mse(y_right) * y_right.size
        den = y_left.size + y_right.size
        return num / den

    def _best_split(self, X: np.ndarray, y: np.ndarray) -> tuple[int, float]:
        """Find the best split for a node."""
        node_size = y.size
        if node_size < self.min_samples_split:
            return None, None
        node_mse = self._mse(y)
        best_mse = node_mse
        best_idx, best_thr = None, None
        for idx in range(self.n_features_):
            thresholds = np.unique(X[:, idx])
            for thr in thresholds:
                left = y[X[:, idx] <= thr]
                right = y[X[:, idx] > thr]

                if left.size == 0 or right.size == 0:
                    continue

                weighted_mse = self._weighted_mse(left, right)
                if weighted_mse < best_mse:
                    best_mse = weighted_mse
                    best_idx = idx
                    best_thr = thr

        return best_idx, best_thr

    def _split_node(self, X: np.ndarray, y: np.ndarray, depth: int = 0) -> Node:
        """Split a node and return the resulting left and right child nodes."""
        if depth == self.max_depth or y.size < self.min_samples_split:
            return Node(value=int(round(np.mean(y))), mse=self._mse(y), n_samples=y.size)

        best_idx, best_thr = self._best_split(X, y)
        if best_thr is None:
            return None

        left_idx = X[:, best_idx] <= best_thr
        right_idx = X[:, best_idx] > best_thr

        left_child = self._split_node(X[left_idx], y[left_idx], depth + 1)
        right_child = self._split_node(X[right_idx], y[right_idx], depth + 1)

        return Node(feature=best_idx, threshold=best_thr, n_samples=y.size, left=left_child, right=right_child,
                    value=int(round(np.mean(y))),
                    mse=self._mse(y))
    def as_json(self) -> str:
        """Return the decision tree as a JSON string."""
        return json.dumps(self._as_json(self.tree_), indent=4)

    def _as_json(self, node: Node) -> dict:
        if node is None:
            return None  # JSON-совместимая строка для None

        if node.left is None and node.right is None:
            return {
                'value': int(node.value) if node.value is not None else None,
                'n_samples': int(node.n_samples) if node.n_samples is not None else None,
                'mse': round(float(node.mse), 2) if node.mse is not None else None
            }

        return {
            'feature': int(node.feature) if node.feature is not None else None,
            'threshold': int(node.threshold) if node.threshold is not None else None,
            'n_samples': int(node.n_samples) if node.n_samples is not None else None,
            'mse': round(float(node.mse), 2) if node.mse is not None else None,
            'left': self._as_json(node.left),
            'right': self._as_json(node.right)
        }
    def _predict_one_sample(self, features: np.ndarray) -> int:
        """Predict the target value of a single sample."""
        node = self.tree_
        while node.left or node.right:
            if features[node.feature] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
    Predict regression target for X.

    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        The input samples.

    Returns
    -------
    y : array of shape (n_samples,)
        The predicted values.
    """
    # YOUR CODE HERE
        return np.array([self._predict_one_sample(sample) for sample in X])


In [19]:
X_train = np.array([
    [76, 32181, 3, 0, 1, 814, 28, 1, 142434, 1770],
    [69, 52789, 8, 1, 0, 501, 28, 1, 120887, 1590],
    [19, 70535, 1, 0, 1, 325, 26, 1, 188766, 810],
    [31, 85271, 1, 0, 1, 525, 29, 1, 406792, 330],
    [18, 19974, 2, 0, 1, 618, 34, 1, 155240, 1560],
    [51, 74128, 3, 0, 1, 551, 14, 0, 257944, 420],
    [67, 34922, 10, 1, 1, 657, 19, 0, 207532, 240],
    [27, 54154, 1, 0, 1, 740, 38, 1, 229763, 660],
    [61, 76998, 8, 0, 0, 869, 35, 1, 147957, 90],
    [61, 41396, 5, 0, 0, 636, 6, 0, 483916, 120]
])

y_train = np.array([0, 7, 0, 0, 43, 0, 3, 0, 0, 12])  # Целевая переменная - 'delay_days'

tree = DecisionTreeRegressor(max_depth=3)
tree.fit(X_train, y_train)
print(tree.as_json())


{
    "feature": 0,
    "threshold": 18.0,
    "n_samples": 10,
    "value": 6,
    "mse": 162.85,
    "left": {
        "feature": null,
        "threshold": null,
        "n_samples": 1,
        "value": 43,
        "mse": 0.0,
        "left": "null",
        "right": "null"
    },
    "right": {
        "feature": 6,
        "threshold": 6.0,
        "n_samples": 9,
        "value": 2,
        "mse": 16.469135802469136,
        "left": {
            "feature": null,
            "threshold": null,
            "n_samples": 1,
            "value": 12,
            "mse": 0.0,
            "left": "null",
            "right": "null"
        },
        "right": {
            "feature": 8,
            "threshold": 120887.0,
            "n_samples": 8,
            "value": 1,
            "mse": 5.6875,
            "left": {
                "feature": null,
                "threshold": null,
                "n_samples": 1,
                "value": 7,
                "mse": 0.0,
              

Визуализация дерева
Дерево построили, теперь визуализируем его. Будет полезно визуально посмотреть, какое дерево у нас получилось. Особенно для глубоких деревьев.

Задание
Вам необходимо реализовать метод as_json(). Метод переводит дерево в строку в формате JSON.

Алгоритм отображения дерева в JSON формате тоже будет рекурсивным. Он аналогичен алгоритму построения дерева. Поэтому удобно будет реализовать еще один внутренний метод, например, _as_json(), который будет вызываться рекурсивно.

Сформировать_json (Node):
    Если это лист, формируем json для листа.
        Останавливаемся.
    Иначе
        Формируем json для текущей ноды
        Сформировать_json (Left)
        Сформировать_json (Right)

In [21]:
def as_json(self) -> str:
    """Return the decision tree as a JSON string."""
    return json.dumps(self._as_json(self.tree_), indent=4)

def _as_json(self, node: Node) -> dict:
    if node is None:
        return None  # JSON-совместимая строка для None

    if node.left is None and node.right is None:
        return {
                'value': int(node.value) if node.value is not None else None,
                'n_samples': int(node.n_samples) if node.n_samples is not None else None,
                'mse': round(float(node.mse), 2) if node.mse is not None else None
            }

    return {
            'feature': int(node.feature) if node.feature is not None else None,
            'threshold': int(node.threshold) if node.threshold is not None else None,
            'n_samples': int(node.n_samples) if node.n_samples is not None else None,
            'mse': round(float(node.mse), 2) if node.mse is not None else None,
            'left': self._as_json(node.left),
            'right': self._as_json(node.right)
        }